<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 811ms/step - loss: 0.6715 - accuracy: 0.5681 - val_loss: 0.6573 - val_accuracy: 0.8049
Epoch 2/10
26/26 [==============================] - 20s 773ms/step - loss: 0.6312 - accuracy: 0.6886 - val_loss: 0.6142 - val_accuracy: 0.8098
Epoch 3/10
26/26 [==============================] - 20s 769ms/step - loss: 0.5886 - accuracy: 0.7591 - val_loss: 0.5447 - val_accuracy: 0.8878
Epoch 4/10
26/26 [==============================] - 20s 766ms/step - loss: 0.5134 - accuracy: 0.8041 - val_loss: 0.4711 - val_accuracy: 0.8829
Epoch 5/10
26/26 [==============================] - 20s 762ms/step - loss: 0.4672 - accuracy: 0.8114 - val_loss: 0.3822 - val_accuracy: 0.8976
Epoch 6/10
26/26 [==============================] - 20s 761ms/step - loss: 0.3771 - accuracy: 0.8783 - val_loss: 0.3058 - val_accuracy: 0.9268
Epoch 7/10
26/26 [==============================] - 20s 769ms/step - loss: 0.3166 - accuracy: 0.9027 - val_loss: 0.2445 - val_accuracy: 0.9366

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.67.h5
26/26 - 20s - loss: 0.6743 - accuracy: 0.5827 - val_loss: 0.6653 - val_accuracy: 0.5854
Epoch 2/5

Epoch 00002: saving model to weights.02-0.62.h5
26/26 - 20s - loss: 0.6333 - accuracy: 0.6800 - val_loss: 0.6207 - val_accuracy: 0.7659
Epoch 3/5

Epoch 00003: saving model to weights.03-0.67.h5
26/26 - 20s - loss: 0.6060 - accuracy: 0.6898 - val_loss: 0.6717 - val_accuracy: 0.5366
Epoch 4/5

Epoch 00004: saving model to weights.04-0.56.h5
26/26 - 20s - loss: 0.5813 - accuracy: 0.6983 - val_loss: 0.5620 - val_accuracy: 0.7366
Epoch 5/5

Epoch 00005: saving model to weights.05-0.49.h5
26/26 - 20s - loss: 0.5082 - accuracy: 0.7798 - val_loss: 0.4866 - val_accuracy: 0.8000


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6801 - accuracy: 0.5316 - val_loss: 0.6928 - val_accuracy: 0.4341


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6634 - accuracy: 0.6290 - val_loss: 0.6423 - val_accuracy: 0.7073
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6196 - accuracy: 0.6886 - val_loss: 0.5891 - val_accuracy: 0.6488


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 22s - loss: 0.6719 - accuracy: 0.6083 - val_loss: 0.7115 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 20s - loss: 0.6283 - accuracy: 0.6715 - val_loss: 0.6415 - val_accuracy: 0.5366
Epoch 3/50
26/26 - 19s - loss: 0.5830 - accuracy: 0.7275 - val_loss: 0.5427 - val_accuracy: 0.8098
Epoch 4/50
26/26 - 20s - loss: 0.5305 - accuracy: 0.7737 - val_loss: 0.4938 - val_accuracy: 0.7854
Epoch 5/50
26/26 - 20s - loss: 0.4773 - accuracy: 0.7871 - val_loss: 0.5765 - val_accuracy: 0.6146
Epoch 6/50
26/26 - 20s - loss: 0.3893 - accuracy: 0.8528 - val_loss: 0.4159 - val_accuracy: 0.8244
Epoch 7/50
26/26 - 19s - loss: 0.3174 - accuracy: 0.8844 - val_loss: 0.3607 - val_accuracy: 0.8390
Epoch 8/50
26/26 - 20s - loss: 0.2894 - accuracy: 0.8856 - val_loss: 0.2507 - val_accuracy: 0.9073
Epoch 9/50
26/26 - 20s - loss: 0.2165 - accuracy: 0.9246 - val_loss: 0.2211 - val_accuracy: 0.9024
Epoch 10/50
26/26 - 21s - loss: 0.1963 - accuracy: 0.9331 - val_loss: 0.1377 - val_accuracy: 0.9854
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 758ms/step - loss: 0.6617 - accuracy: 0.5925 - val_loss: 0.7007 - val_accuracy: 0.4829
Epoch 2/5
26/26 [==============================] - 19s 739ms/step - loss: 0.5913 - accuracy: 0.6934 - val_loss: 0.6415 - val_accuracy: 0.5512
Epoch 3/5
26/26 [==============================] - 22s 834ms/step - loss: 0.5369 - accuracy: 0.7397 - val_loss: 0.4806 - val_accuracy: 0.8146
Epoch 4/5
26/26 [==============================] - 20s 773ms/step - loss: 0.4574 - accuracy: 0.8017 - val_loss: 0.4195 - val_accuracy: 0.8439
Epoch 5/5
26/26 [==============================] - 20s 765ms/step - loss: 0.4206 - accuracy: 0.8273 - val_loss: 0.3408 - val_accuracy: 0.8585


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.592457,0.662492,0.482927,0.700721
1,1,0.693431,0.591382,0.551219,0.641513
2,2,0.739659,0.536992,0.814634,0.480568
3,3,0.801703,0.457298,0.843902,0.419472
4,4,0.827251,0.421275,0.858537,0.340835


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 31s 1s/step - loss: 0.6800 - accuracy: 0.5547 - val_loss: 0.6698 - val_accuracy: 0.7463

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 22s 831ms/step - loss: 0.6620 - accuracy: 0.6764 - val_loss: 0.6678 - val_accuracy: 0.5756

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6501 - accuracy: 0.6727 - val_loss: 0.6652 - val_accuracy: 0.5561

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 769ms/step - loss: 0.6451 - accuracy: 0.6618 - val_loss: 0.6618 - val_accuracy: 0.5707

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 765ms/step - loss: 0.6422

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 23s 888ms/step - loss: 0.6649 - accuracy: 0.5572 - val_loss: 0.6862 - val_accuracy: 0.4780
Epoch 2/50
26/26 [==============================] - 21s 811ms/step - loss: 0.6279 - accuracy: 0.6764 - val_loss: 0.6362 - val_accuracy: 0.5756
Epoch 3/50
26/26 [==============================] - 20s 774ms/step - loss: 0.5469 - accuracy: 0.7372 - val_loss: 0.5679 - val_accuracy: 0.6585
Epoch 4/50
26/26 [==============================] - 20s 770ms/step - loss: 0.5212 - accuracy: 0.7506 - val_loss: 0.5401 - val_accuracy: 0.6488
Epoch 5/50
26/26 [==============================] - 24s 918ms/step - loss: 0.4591 - accuracy: 0.7981 - val_loss: 0.3831 - val_accuracy: 0.9171
Epoch 6/50
26/26 [==============================] - 20s 781ms/step - loss: 0.3839 - accuracy: 0.8516 - val_loss: 0.3290 - val_accuracy: 0.9073
Epoch 7/50
26/26 [==============================] - 20s 781ms/step - loss: 0.3395 - accuracy: 0.8759 - val_loss: 0.3166 - val_accuracy: 0.8976

In [ ]:
%tensorboard --logdir log_dir